In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from collections import Counter
from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.metrics import confusion_matrix

In [10]:
engine = create_engine('sqlite:///../data/titanic.db')
sqlite_connection = engine.connect()
pd.read_sql('SELECT * FROM sqlite_schema WHERE type="table"', con=sqlite_connection)

,type,name,tbl_name,rootpage,sql
0,table,tbl_passengers,tbl_passengers,2,"CREATE TABLE tbl_passengers (\n\tpid BIGINT, \..."
1,table,tbl_targets,tbl_targets,35,"CREATE TABLE tbl_targets (\n\tpid BIGINT, \n\t..."


In [11]:
df = pd.read_sql('SELECT * FROM tbl_passengers', con=sqlite_connection)

In [17]:
query='SELECT tbl_passengers.*,tbl_targets.is_survived FROM tbl_passengers JOIN tbl_targets ON tbl_passengers.pid=tbl_targets.pid'

In [23]:
df = pd.read_pickle('../data/passengers.pkl')

In [25]:
next(iter(df.itertuples()))

Pandas(Index=0, pid=0, pclass=1.0, name='Allen, Miss. Elisabeth Walton', sex='female', age=29.0, sibsp=0.0, parch=0.0, ticket='24160', fare=211.3375, cabin='B5', embarked='S', boat='2', body=nan, _14='St Louis, MO', is_survived=1)

In [26]:
df[:3]

,pid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,is_survived
0,0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",1
1,1,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",1
2,2,1.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON",0


In [3]:
np.random.seed(42)

df = pd.read_sql('SELECT * FROM tbl_passengers', con=sqlite_connection)

targets = pd.read_sql('SELECT * FROM tbl_targets', con=sqlite_connection)

# df, targets = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

# parch = Parents/Children, sibsp = Siblings/Spouses
df['family_size'] = df['parch'] + df['sibsp']
df['is_alone'] = [1 if family_size==1 else 0 for family_size in df['family_size']]

df['title'] = [name.split(',')[1].split('.')[0].strip() for name in df['name']]
rare_titles = {k for k,v in Counter(df['title']).items() if v < 10}
df['title'] = ['rare' if title in rare_titles else title for title in df['title']]

df = df[[
    'pclass', 'sex', 'age', 'ticket', 'family_size',
    'fare', 'embarked', 'is_alone', 'title'
]]

targets = [int(v) for v in targets['is_survived']]
X_train, X_test, y_train, y_test = train_test_split(df, targets, stratify=targets, test_size=0.2)

In [9]:
rare_titles

{'Capt',
 'Col',
 'Don',
 'Dona',
 'Dr',
 'Jonkheer',
 'Lady',
 'Major',
 'Mlle',
 'Mme',
 'Ms',
 'Rev',
 'Sir',
 'the Countess'}

In [6]:
targets = pd.read_sql('SELECT * FROM tbl_targets', con=sqlite_connection)

In [8]:
pd.read_pickle('../data/targets.pkl')

,pid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,2,1.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,3,1.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,4,1.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1304,3.0,"Zabour, Miss. Hileni",female,14.5000,1.0,0.0,2665,14.4542,None,C,None,328.0,None
1305,1305,3.0,"Zabour, Miss. Thamine",female,NaN,1.0,0.0,2665,14.4542,None,C,None,NaN,None
1306,1306,3.0,"Zakarian, Mr. Mapriededer",male,26.5000,0.0,0.0,2656,7.2250,None,C,None,304.0,None
1307,1307,3.0,"Zakarian, Mr. Ortin",male,27.0000,0.0,0.0,2670,7.2250,None,C,None,NaN,None


In [ ]:
X_train_categorical = X_train[['embarked', 'sex', 'pclass', 'title', 'is_alone']]
X_test_categorical = X_test[['embarked', 'sex', 'pclass', 'title', 'is_alone']]

oneHotEncoder = OneHotEncoder(handle_unknown='ignore', sparse=False).fit(X_train_categorical)
X_train_categorical_one_hot = oneHotEncoder.transform(X_train_categorical)
X_test_categorical_one_hot = oneHotEncoder.transform(X_test_categorical)

In [ ]:
X_train_numerical = X_train[['age', 'fare', 'family_size']]
X_test_numerical = X_test[['age', 'fare', 'family_size']]
knnImputer = KNNImputer(n_neighbors=5).fit(X_train_numerical)
X_train_numerical_imputed = knnImputer.transform(X_train_numerical)
X_test_numerical_imputed = knnImputer.transform(X_test_numerical)

In [ ]:
robustScaler = RobustScaler().fit(X_train_numerical_imputed)
X_train_numerical_imputed_scaled = robustScaler.transform(X_train_numerical_imputed)
X_test_numerical_imputed_scaled = robustScaler.transform(X_test_numerical_imputed)

In [ ]:
X_train_processed = np.hstack((X_train_categorical_one_hot, X_train_numerical_imputed_scaled))
X_test_processed = np.hstack((X_test_categorical_one_hot, X_test_numerical_imputed_scaled))

In [ ]:
model = LogisticRegression(random_state=0).fit(X_train_processed, y_train)
y_train_estimation = model.predict(X_train_processed)
y_test_estimation = model.predict(X_test_processed)

In [ ]:
cm_train = confusion_matrix(y_train, y_train_estimation)

In [ ]:
cm_test = confusion_matrix(y_test, y_test_estimation)

In [ ]:
cm_train

In [ ]:
cm_test

In [ ]:
def do_test(filename, data):
    if not os.path.isfile(filename):
        pickle.dump(data, open(filename, 'wb'))
    truth = pickle.load(open(filename, 'rb'))
    try:
        np.testing.assert_almost_equal(data, truth)
        print(f'{filename} test passed')
    except AssertionError as ex:
        print(f'{filename} test failed {ex}')
    
do_test('../data/cm_test.pkl', cm_test)
do_test('../data/cm_train.pkl', cm_train)
do_test('../data/X_train_processed.pkl', X_train_processed)
do_test('../data/X_test_processed.pkl', X_test_processed)


In [ ]:
def do_pandas_test(filename, data):
    if not os.path.isfile(filename):
        data.to_pickle(filename)
    truth = pd.read_pickle(filename)
    try:
        pd.testing.assert_frame_equal(data, truth)
        print(f'{filename} pandas test passed')
    except AssertionError as ex:
        print(f'{filename} pandas test failed {ex}')
        
# df['title'] = ['asd' for v in df['title']]
do_pandas_test('../data/df.pkl', df)